In [18]:
import pandas as pd
import numpy as np
import re
import random
import pymysql
from pymysql.err import ProgrammingError
import math


In [2]:
host = "127.0.0.1"
port = 3306
username = "root"
password = "123456"
database = "analysis_sys_db"
charset = "utf8mb4"

In [3]:
conn = pymysql.connect(
    host=host,
    port=port,
    user=username,
    password=password,
    database=database,
    charset=charset
)

In [4]:
monitor_work_df = pd.read_sql("select id, name, category, labels, postTime from monitor_work", conn)
monitor_work_df

,id,name,category,labels,postTime
0,1,西游记,影视,动作 冒险 古装,2020-07-04
1,2,流浪地球1,影视,科幻 冒险 灾难,2021-05-15
2,4,流浪地球2,影视,科幻 冒险 灾难,2023-01-22
3,5,李子柒短视频,影视,现实 生活,2021-02-18
4,6,敢教日月换新天 (2021),影视,纪录片,2021-06-20
...,...,...,...,...,...
152,178,东北告别天团,影视,喜剧,2022-04-22
153,179,倒数说爱你,影视,爱情 奇幻,2023-04-28
154,180,纸人回魂,影视,悬疑 恐怖,2023-01-06
155,181,特级英雄黄继光,影视,剧情 传记 战争,2022-10-01


In [6]:
all_labels = []
monitor_work_df['labels'].apply(lambda x: all_labels.extend(re.split(' +', x.strip())))
all_labels = set(all_labels)
all_labels = list(all_labels)
all_labels

['诗歌',
 '地域',
 '运动',
 '女性',
 '史诗',
 '犯罪',
 '生存',
 '恐怖',
 '武侠',
 '记忆',
 '喜剧',
 '灾难',
 '爱情',
 '动画',
 '亲情',
 '音乐',
 '惊悚',
 '纪录片',
 '农村',
 '剧情',
 '短片',
 '哲思',
 '小说',
 '友情',
 '奇幻',
 '儿童',
 '战争',
 '家庭',
 '散文',
 '悬疑',
 '动作',
 '冒险',
 '悲剧',
 '情感',
 '传记',
 '杂文',
 '寓言',
 '古装',
 '文学',
 '人性',
 '生活',
 '游戏',
 '命运',
 '人生',
 '科幻',
 '神话',
 '现实',
 '历史',
 '同性',
 '饮食']

In [7]:
label_num = len(all_labels)
label_num

50

In [8]:
# 提取所有的作品id
all_work_names = monitor_work_df['id'].values
data_mat = []
for i in range(len(all_work_names)):
    vector = [0] * label_num
    label_names = re.split(' +', monitor_work_df.iloc[i, 3].strip())
    for label in label_names:
        if len(label.strip()) == 0:
            continue
        loc = all_labels.index(label)
        vector[loc] = 1
    data_mat.append(vector)


# 各个作品的标签矩阵
work_labels_df = pd.DataFrame(data=data_mat, index=all_work_names, columns=all_labels)
work_labels_df

,诗歌,地域,运动,女性,史诗,犯罪,生存,恐怖,武侠,记忆,...,生活,游戏,命运,人生,科幻,神话,现实,历史,同性,饮食
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
user_record_df = pd.read_sql("select * from user_recent_record;", conn)
user_record_df

,id,userId,workId,visitCounts,recentVisit
0,1,26,1,4,2023-09-12 20:12:08
1,2,26,2,3,2023-09-12 20:13:22
2,3,26,4,2,2023-09-12 21:14:26
3,4,26,174,1,2023-09-12 20:56:27
4,5,22,2,5,2023-09-13 18:44:21
5,6,22,1,7,2023-09-13 18:44:40
6,7,22,4,3,2023-09-13 18:44:39
7,8,22,5,3,2023-09-13 18:44:02
8,9,22,7,2,2023-09-13 18:43:22
9,10,22,19,2,2023-09-13 18:44:10


In [10]:
workId_lst = user_record_df['workId'].unique().tolist()
userId_lst = user_record_df['userId'].unique().tolist()
work_num = len(workId_lst)
user_work_data = []
for userId in userId_lst:
    row_ = [0] * work_num
    query_df = user_record_df.query('userId == {}'.format(userId))  # 筛选userId用户的浏览记录
    max_cnt = query_df['visitCounts'].max()  # 用户的最大浏览次数
    for i in query_df.index:
        loc = workId_lst.index(query_df.loc[i, 'workId'])
        row_[loc] = query_df.loc[i, 'visitCounts'] / max_cnt
    user_work_data.append(row_)

user_work_df = pd.DataFrame(data=user_work_data, index=userId_lst, columns=workId_lst)
user_work_df

,1,2,4,174,5,7,19,20,9,21,18
26,1.0,0.750000,0.500000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22,1.0,0.714286,0.428571,0.142857,0.428571,0.285714,0.285714,0.142857,0.142857,0.142857,0.142857


In [11]:
monitor_work_less = monitor_work_df[['id', 'labels']]
monitor_work_less.columns = ['workId', 'labels']
monitor_work_less

,workId,labels
0,1,动作 冒险 古装
1,2,科幻 冒险 灾难
2,4,科幻 冒险 灾难
3,5,现实 生活
4,6,纪录片
...,...,...
152,178,喜剧
153,179,爱情 奇幻
154,180,悬疑 恐怖
155,181,剧情 传记 战争


In [12]:
user_record_df_merge = pd.merge(user_record_df, monitor_work_less, how='inner')
user_record_df_merge

,id,userId,workId,visitCounts,recentVisit,labels
0,1,26,1,4,2023-09-12 20:12:08,动作 冒险 古装
1,6,22,1,7,2023-09-13 18:44:40,动作 冒险 古装
2,2,26,2,3,2023-09-12 20:13:22,科幻 冒险 灾难
3,5,22,2,5,2023-09-13 18:44:21,科幻 冒险 灾难
4,3,26,4,2,2023-09-12 21:14:26,科幻 冒险 灾难
5,7,22,4,3,2023-09-13 18:44:39,科幻 冒险 灾难
6,4,26,174,1,2023-09-12 20:56:27,喜剧 动画 奇幻 冒险
7,14,22,174,1,2023-09-13 18:44:56,喜剧 动画 奇幻 冒险
8,8,22,5,3,2023-09-13 18:44:02,现实 生活
9,9,22,7,2,2023-09-13 18:43:22,剧情 科幻


In [13]:
user_record_df_merge_unique = user_record_df_merge[['workId', 'labels']].drop_duplicates()
user_record_df_merge_unique

,workId,labels
0,1,动作 冒险 古装
2,2,科幻 冒险 灾难
4,4,科幻 冒险 灾难
6,174,喜剧 动画 奇幻 冒险
8,5,现实 生活
9,7,剧情 科幻
10,19,剧情 喜剧 动画 科幻
11,20,纪录片 短片
12,9,剧情 历史 战争
13,21,历史


In [14]:
df_data = []
for i in user_record_df_merge_unique.index:
    row_ = [0]*label_num
    labels = re.split(' +', user_record_df_merge_unique.loc[i, 'labels'].strip())
    for label in labels:
        loc = all_labels.index(label)
        row_[loc] = 1
    df_data.append(row_)

all_user_saw_work_label_df = pd.DataFrame(data=df_data, 
                                          index=user_record_df_merge_unique['workId'], columns=all_labels)
all_user_saw_work_label_df

,诗歌,地域,运动,女性,史诗,犯罪,生存,恐怖,武侠,记忆,...,生活,游戏,命运,人生,科幻,神话,现实,历史,同性,饮食
workId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [56]:
# 创建作品画像
# 参数说明：
# items_profiles = {item1:{'label1':1, 'label2': 0, 'label3': 0, ...}, item2:{...}...}
def createItemsProfiles(data_array, labels_names, items_names):

    items_profiles = {}

    for i in range(len(items_names)):

        items_profiles[items_names[i]] = {}

        for j in range(len(labels_names)):
            items_profiles[items_names[i]][labels_names[j]] = data_array[i][j]

    return items_profiles

# 创建用户画像
# 参数说明：
# data_array: 所有用户对于其所看过的节目的评分矩阵 data_array = [[2, 0, 0, 1.1, ...], [0, 0, 1.1, ...], ...]
# users_profiles = {user1:{'label1':1.1, 'label2': 0.5, 'label3': 0.0, ...}, user2:{...}...}
def createUsersProfiles(data_array, users_names, items_names, labels_names, items_profiles):

    users_profiles = {}

    # 计算每个用户对所看过的所有节目的平均隐性评分
    # users_average_scores_list = [1.2, 2.2, 4.3,...]
    users_average_scores_list = []

    # 统计每个用户所看过的节目（不加入隐性评分信息）
    # items_users_saw = {user1:[item1, item3, item5], user2:[...],...}
    items_users_saw = {}

    # 统计每个用户所看过的节目及评分
    # items_users_saw_scores = {user1:[[item1, 1.1], [item2, 4.1]], user2:...}
    items_users_saw_scores = {}

    for i in range(len(users_names)):

        items_users_saw_scores[users_names[i]] = []
        items_users_saw[users_names[i]] = []
        count = 0
        sum = 0.0

        for j in range(len(items_names)):

            # 用户对该节目隐性评分为正，表示真正看过该节目
            if data_array[i][j] > 0:
                items_users_saw[users_names[i]].append(items_names[j])
                items_users_saw_scores[users_names[i]].append([items_names[j], data_array[i][j]])
                count += 1
                sum += data_array[i][j]

        if count == 0:
            users_average_scores_list.append(0)
        else:
            users_average_scores_list.append(sum / count)

    for i in range(len(users_names)):

        users_profiles[users_names[i]] = {}

        for j in range(len(labels_names)):
            count = 0
            score = 0.0

            for item in items_users_saw_scores[users_names[i]]:

                # 参数：
                # 用户user1对于类型label1的隐性评分: user1_score_to_label1
                # 用户user1对于其看过的含有类型label1的节目item i 的评分: score_to_item i
                # 用户user1对其所看过的所有节目的平均评分: user1_average_score
                # 用户user1看过的节目总数: items_count

                # 公式： user1_score_to_label1 = Sigma(score_to_item i - user1_average_score)/items_count

                # 该节目含有特定标签labels_names[j]
                if items_profiles[item[0]][labels_names[j]] > 0:
                    score += (item[1] - users_average_scores_list[i])
                    count += 1

            # 如果求出的值太小，直接置0
            if abs(score) < 1e-6:
                score = 0.0
            if count == 0:
                result = 0.0
            else:
                result = score / count

            users_profiles[users_names[i]][labels_names[j]] = result

    return (users_profiles, items_users_saw)


# 计算用户画像向量与作品画像向量的距离（相似度）
# 向量相似度计算公式：
# cos(user, item) = sigma_ui/sqrt(sigma_u * sigma_i)

# 参数说明：
# user_profile: 某一用户user的画像 user = {'label1':1.1, 'label2': 0.5, 'label3': 0.0, ...}
# item: 某一节目item的画像 item = {'label1':1, 'label2': 0, 'label3': 0, ...}
# labels_names: 所有类型名
def calCosDistance(user, item, labels_names):

    sigma_ui = 0.0
    sigma_u = 0.0
    sigma_i = 0.0

    for label in labels_names:
        sigma_ui += user[label] * item[label]
        sigma_u += (user[label] * user[label])
        sigma_i += (item[label] * item[label])

    if sigma_u == 0.0 or sigma_i == 0.0:  # 若分母为0，相似度为0
        return 0

    return sigma_ui/math.sqrt(sigma_u * sigma_i)


# 基于内容的推荐算法：
# 借助特定某个用户user的画像user_profile和备选推荐节目集的画像items_profiles，通过计算向量之间的相似度得出推荐节目集

# 参数说明：
# user_profile: 某一用户user的画像 user_profile = {'label1':1.1, 'label2': 0.5, 'label3': 0.0, ...}
# items_profiles: 备选推荐节目集的节目画像: items_profiles = {item1:{'label1':1, 'label2': 0, 'label3': 0}, item2:{...}...}
# items_names: 备选推荐节目集中的所有节目名
# labels_names: 所有类型名
# items_user_saw: 用户user看过的节目

def contentBased(user_profile, items_profiles, items_names, labels_names, items_user_saw):

    # 对于用户user的推荐节目集为 recommend_items = [[节目名, 该节目画像与该用户画像的相似度], ...]
    recommend_items = []

    for i in range(len(items_names)):
        # 从备选推荐节目集中的选择用户user没有看过的节目
        
        if items_names[i] not in items_user_saw:
            recommend_items.append([items_names[i], calCosDistance(user_profile, items_profiles[items_names[i]], labels_names)])
        else:
            randi = random.random()
            if randi > 0.5:
                recommend_items.append([items_names[i], calCosDistance(user_profile, items_profiles[items_names[i]], labels_names)])

    # 将推荐节目集按相似度降序排列
    recommend_items.sort(key=lambda item: item[1], reverse=True)

    return recommend_items

# 输出推荐给该用户的节目列表
# max_num:最多输出的推荐节目数
def printRecommendedItems(recommend_items_sorted, max_num):
    count = 0
    for item, degree in recommend_items_sorted:
        print("节目名：%s， 推荐指数：%f" % (item, degree))
        count += 1
        if count == max_num:
            break

In [59]:
# 用户id列表
all_users_ids = user_record_df['userId'].unique().tolist()
# 所有用户对其看过的作品的评分矩阵  浏览次数作为隐性评分
data_array1 = user_work_df.values
# 所有用户浏览过的作品
works_user_saw_names1 = user_work_df.columns.tolist()

# 所有用户看过的作品标签矩阵
data_array2 = all_user_saw_work_label_df.values
works_user_saw_names2 = all_user_saw_work_label_df.index.tolist()

# 为用户浏览过的作品建立作品画像
works_users_saw_profile = createItemsProfiles(data_array2, all_labels, works_user_saw_names2)

# 建立用户画像和用户浏览过的作品集
(users_profiles, works_users_saw) = createUsersProfiles(data_array1, all_users_ids, 
                                                       works_user_saw_names1, all_labels, works_users_saw_profile)

# 备选作品的标签矩阵
data_array3 = work_labels_df.values
works_to_be_recommended_names = work_labels_df.index.tolist()

# 为备选推荐作品集建立作品画像
works_to_be_recommended_profiles = createItemsProfiles(data_array3, all_labels, works_to_be_recommended_names)

for user in all_users_ids:
    print("对于用户 %s 的推荐节目如下：" % user)
    recommend_works = contentBased(users_profiles[user], works_to_be_recommended_profiles, works_to_be_recommended_names, all_labels, works_users_saw[user])
    printRecommendedItems(recommend_works, 8)

对于用户 26 的推荐节目如下：
节目名：1， 推荐指数：0.516398
节目名：24， 推荐指数：0.316228
节目名：105， 推荐指数：0.316228
节目名：146， 推荐指数：0.316228
节目名：149， 推荐指数：0.316228
节目名：169， 推荐指数：0.316228
节目名：8， 推荐指数：0.258199
节目名：17， 推荐指数：0.258199
对于用户 22 的推荐节目如下：
节目名：146， 推荐指数：0.464683
节目名：149， 推荐指数：0.414895
节目名：169， 推荐指数：0.414895
节目名：105， 推荐指数：0.342980
节目名：111， 推荐指数：0.338761
节目名：24， 推荐指数：0.282129
节目名：8， 推荐指数：0.280042
节目名：125， 推荐指数：0.280042


In [72]:
# 借助pearson相关系数进行修正后的余弦相似度计算公式，计算两个用户之间的相似度
# 记  sim(user1, user2) = sigma_xy /sqrt(sigma_x * sigma_y)
# user1和user2都表示为[[节目名称,隐性评分], [节目名称,隐性评分]],如user1 = [['节目一', 3.2], ['节目四', 0.2], ['节目八', 6.5], ...]

def calCosDistByPearson(user1, user2):
    x = 0.0
    y = 0.0

    sigma_xy = 0.0
    sigma_x = 0.0
    sigma_y = 0.0

    for item in user1:
        x += item[1]

    # user1对其看过的所有节目的平均评分
    average_x = x / len(user1)

    for item in user2:
        y += item[1]

    # user2对其看过的所有节目的平均评分
    average_y = y / len(user2)

    for item1 in user1:
        for item2 in user2:
            if item1[0] == item2[0]:  # 对user1和user2都共同看过的节目才考虑进去
                sigma_xy += (item1[1] - average_x) * (item2[1] - average_y)
                sigma_x += (item1[1] - average_x) * (item1[1] - average_x)
                sigma_y += (item2[1] - average_y) * (item2[1] - average_y)

    if sigma_x == 0.0 or sigma_y == 0.0:  # 若分母为0，相似度为0
        return 0

    return sigma_xy/math.sqrt(sigma_x * sigma_y)


# 创建所有用户的观看信息（包含隐性评分信息）,“从用户到节目”
# 格式例子：users_to_items = {用户一:[['节目一', 3.2], ['节目四', 0.2], ['节目八', 6.5]], 用户二: ... }
def createUsersDict(df):
    
    (m, n) = df.shape
    data_array = df.values
    users_names = df.index.tolist()
    items_names = df.columns.tolist()

    users_to_items = {}

    for i in range(len(users_names)):
        user_and_scores_list = []
        for j in range(len(items_names)):
            if data_array[i][j] > 0:
                user_and_scores_list.append([items_names[j], data_array[i][j]])
        users_to_items[users_names[i]] = user_and_scores_list

    return users_to_items

# 创建所有节目被哪些用户观看的字典，也就是创建“从节目到用户”的倒排表items_and_users
# items_to_users = {节目一: [用户一, 用户三], 节目二: ... }
def createItemsDict(df):
    
    (m, n) = df.shape
    data_array = df.values
    users_names = df.index.tolist()
    items_names = df.columns.tolist()
    items_to_users = {}

    for i in range(len(items_names)):
        users_list = []
        for j in range(len(users_names)):
            if data_array[j][i] > 0:
                users_list.append(users_names[j])
        items_to_users[items_names[i]] = users_list

    return items_to_users


# 找出与用户user_name相关的所有用户（即邻居）并依照相似度排序
# neighbors_distance = [[用户名, 相似度大小], [...], ...] = [['用户四', 0.78],[...], ...]
def findSimilarUsers(users_dict, items_dict, user_name):

    neighbors = []   # neighbors表示与该用户看过相同节目的所有用户

    for items in users_dict[user_name]:
        for neighbor in items_dict[items[0]]:
            if neighbor != user_name and neighbor not in neighbors:
                neighbors.append(neighbor)

    # 计算该用户与其所有邻居的相似度并降序排序
    neighbors_distance = []
    for neighbor in neighbors:
        distance = calCosDistByPearson(users_dict[user_name], users_dict[neighbor])
        neighbors_distance.append([neighbor, distance])

    neighbors_distance.sort(key=lambda item: item[1], reverse=True)

    return neighbors_distance


# 基于用户的协同过滤算法
# K为邻居个数，是一个重要参数，参数调优时使用
def userCF(user_name, users_dict, items_dict, K, all_items_names_to_be_recommend):

    # recommend_items = {节目名：某个看过该节目的该用户user_name的邻居与该用户的相似度, ...}
    recommend_items = {}
    # 将上面的recommend_items转换成列表形式并排序为recommend_items_sorted = [[节目一, 该用户对节目一的感兴趣程度],[...], ...]
    recommend_items_sorted = []

    # 用户user_name看过的节目
    items_user_saw = []
    for item in users_dict[user_name]:
        items_user_saw.append(item[0])

    # 找出与该用户相似度最大的K个用户(邻居)
    similar_users = findSimilarUsers(users_dict, items_dict, user_name)
    if len(similar_users) < K:
        k_similar_user = similar_users
    else:
        k_similar_user = similar_users[:K]

    # 得出对该用户的推荐节目集
    for user in k_similar_user:
        for item in users_dict[user[0]]:
            # 该用户user_name没有看过的节目才添加进来，才可以推荐给该用户
            if item[0] not in items_user_saw:
                # 而且该节目必须是在备选推荐节目集中
                if item[0] in all_items_names_to_be_recommend:
                    if item[0] not in recommend_items:
                        # recommend_items是一个字典。第一次迭代中，表示将第一个邻居用户与该用户的相似度加到节目名上，后续迭代如果有其他邻居用户也看过该节目，
                        # 也将其与该用户的相似度加到节目名上，迭代的结果就是该用户对该节目的感兴趣程度
                        recommend_items[item[0]] = user[1]

                    else:
                        # 如果某个节目有k个邻居用户看过，则将这k个邻居用户与该用户的相似度相加，得到该用户对某个节目的感兴趣程度
                        recommend_items[item[0]] += user[1]

    for key in recommend_items:
        recommend_items_sorted.append([key, recommend_items[key]])

    # 对推荐节目集按用户感兴趣程度降序排序
    recommend_items_sorted.sort(key=lambda item: item[1], reverse=True)

    return recommend_items_sorted

# 输出推荐给该用户的节目列表
# max_num:最多输出的推荐节目数
def printRecommendItems(recommend_items_sorted, max_num):
    count = 0
    for item, degree in recommend_items_sorted:
        print("节目名：%s， 推荐指数：%f" % (item, degree))
        count += 1
        if count == max_num:
            break

In [74]:
# 用户id列表
all_users_ids = user_record_df['userId'].unique().tolist()
# 所有用户对其看过的作品的评分矩阵  浏览次数作为隐性评分
data_array1 = user_work_df.values
# 所有用户浏览过的作品
works_user_saw_names1 = user_work_df.columns.tolist()

# users_dict = {用户一:[['节目一', 3.2], ['节目四', 0.2], ['节目八', 6.5]], 用户二: ... }
users_dict = createUsersDict(user_work_df)
# items_dict = {节目一: [用户一, 用户三], 节目二: [...], ... }
items_dict = createItemsDict(user_work_df)


# 所有用户看过的作品标签矩阵
data_array2 = all_user_saw_work_label_df.values
works_user_saw_names2 = all_user_saw_work_label_df.index.tolist()

# 为用户浏览过的作品建立作品画像
works_users_saw_profile = createItemsProfiles(data_array2, all_labels, works_user_saw_names2)

# 建立用户画像和用户浏览过的作品集
(users_profiles, works_users_saw) = createUsersProfiles(data_array1, all_users_ids, 
                                                       works_user_saw_names1, all_labels, works_users_saw_profile)

# 备选作品的标签矩阵
data_array3 = work_labels_df.values
works_to_be_recommended_names = work_labels_df.index.tolist()
# 为备选推荐作品集建立作品画像
works_to_be_recommended_profiles = createItemsProfiles(data_array3, all_labels, works_to_be_recommended_names)


# 两种推荐算法后融合，也就是将两种推荐算法对某个用户分别产生的两个推荐节目集按不同比例混合，得出最后的对该用户的推荐结果
# 对于每个用户推荐topN个节目,在两种推荐算法产生的推荐集中分别选取比例为w1和w2的推荐结果,CB占w1, userCF占w2
# w1 + w2 = 1 且 w1 * topN + w2 * topN = topN
topN = 10

w1 = 0.7
w2 = 0.3

# 从CB的推荐集中选出前topW1项
topW1 = int(w1 * topN)

# 从userCF的推荐集中选出前topW2项
topW2 = topN-topW1

for user in all_users_ids:

    # 对于用户user的最终混合推荐节目集
    recommend_items = []

    # CB
    # recommend_items1 =  [[节目名, 该节目与该用户user画像的相似度], ...]
    recommend_items1 = contentBased(users_profiles[user], works_to_be_recommended_profiles, works_to_be_recommended_names, all_labels, works_users_saw[user])
    len1 = len(recommend_items1)

    if len1 <= topW1:
        recommend_items = recommend_items + recommend_items1
    else:
        recommend_items = recommend_items + recommend_items1[:topW1]


    # userCF
    # recommend_item2 = [[节目名, 该用户user对该节目的感兴趣程度],...]
    recommend_items2 = userCF(user, users_dict, items_dict, 2, works_to_be_recommended_names)
    len2 = len(recommend_items2)

    if len2 <= topW2:
        recommend_items = recommend_items + recommend_items2
    else:
        recommend_items = recommend_items + recommend_items2[:topW2]

    # 将推荐结果按推荐指数降序排序
    recommend_items.sort(key=lambda item: item[1], reverse=True)

    print("对于用户 %s 的推荐节目如下" % user)
    printRecommendItems(recommend_items, 10)
    print()

对于用户 26 的推荐节目如下
节目名：5， 推荐指数：0.822638
节目名：7， 推荐指数：0.822638
节目名：19， 推荐指数：0.822638
节目名：24， 推荐指数：0.316228
节目名：105， 推荐指数：0.316228
节目名：146， 推荐指数：0.316228
节目名：149， 推荐指数：0.316228
节目名：169， 推荐指数：0.316228
节目名：8， 推荐指数：0.258199
节目名：17， 推荐指数：0.258199

对于用户 22 的推荐节目如下
节目名：146， 推荐指数：0.464683
节目名：149， 推荐指数：0.414895
节目名：169， 推荐指数：0.414895
节目名：105， 推荐指数：0.342980
节目名：111， 推荐指数：0.338761
节目名：24， 推荐指数：0.282129
节目名：8， 推荐指数：0.280042

